In [51]:
!pip install tensorflow-text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [52]:
df= pd.read_csv("spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [53]:
df.groupby('Category').describe()

Message                                                               
           count unique                                                top freq
Category                                                                       
ham         4825   4516                             Sorry, I'll call later   30
spam         747    641  Please call our customer service representativ...    4

In [54]:
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()


,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['spam'], stratify=df['spam'])

In [56]:
y_train.value_counts()

0    3619
1     560
Name: spam, dtype: int64

In [57]:
y_test.value_counts()

0    1206
1     187
Name: spam, dtype: int64

In [58]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [59]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']


In [60]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [61]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [62]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
131/131 [==============================] - 1708s 13s/step - loss: 0.3114 - accuracy: 0.8720
Epoch 2/5
131/131 [==============================] - 1775s 14s/step - loss: 0.2270 - accuracy: 0.9036
Epoch 3/5
131/131 [==============================] - 1720s 13s/step - loss: 0.1929 - accuracy: 0.9237
Epoch 4/5
131/131 [==============================] - 1702s 13s/step - loss: 0.1680 - accuracy: 0.9361
Epoch 5/5
131/131 [==============================] - 1698s 13s/step - loss: 0.1552 - accuracy: 0.9462


In [63]:
model.evaluate(X_test, y_test)

44/44 [==============================] - 564s 13s/step - loss: 0.1391 - accuracy: 0.9598


[0.13913358747959137, 0.9597989916801453]

In [64]:
reviews = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

array([[0.6212756 ],
       [0.66421944],
       [0.48550448],
       [0.05397972],
       [0.0181275 ]], dtype=float32)